In [2]:
import pandas as pd

In [1]:
import numpy as np

In [16]:
data = np.load('/mnt/data/DataSet/K-RadarOOC/train/4/radar_pc/rpc_00100.npy')

In [19]:
data.shape

(1632, 4)

In [25]:
data.astype(np.float32)[:,2].max()

5.4

Step 1: Lidar to .bin

In [3]:
from pypcd import pypcd
lidarfiles = glob.glob("/mnt/data/DataSet/K-Radar/1/os2-64/*.pcd")
pcd_data = pypcd.PointCloud.from_path(lidarfiles[0])
pcd_data.get_metadata()

NameError: name 'glob' is not defined

In [42]:
import numpy as np
from pypcd import pypcd
import os.path as osp
def from_pcd_to_bin(file):
    pcd_data = pypcd.PointCloud.from_path(file)
    np_x = (np.array(pcd_data.pc_data['x'], dtype=np.float32)).astype(np.float32)
    np_y = (np.array(pcd_data.pc_data['y'], dtype=np.float32)).astype(np.float32)
    np_z = (np.array(pcd_data.pc_data['z'], dtype=np.float32)).astype(np.float32)
    np_i = (np.array(pcd_data.pc_data['intensity'], dtype=np.float32)).astype(np.float32)/256
    np_ring = (np.array(pcd_data.pc_data['ring'], dtype=np.float32)).astype(np.float32)
    points_bin = np.transpose(np.vstack((np_x, np_y, np_z, np_i,np_ring)))
    filename = file.split("/")[-1]
    seq = file.split("/")[-3]
    new_path = osp.join("/mnt/data/DataSet/K-RadarOOC/train/",seq,"os2-64",filename+'.bin')
    points_bin.tofile(new_path)

In [46]:
import glob
from multiprocess import Pool
from tqdm import tqdm
import os
raw_clips = sorted(os.listdir('/mnt/Kradar/K-Radar'))
# Seq = [1,2,4,6,7,8,9,10,11,12,13,14,16,17,18,19,20,27,56]
def initializer():
    """Prevent tqdm from being multi-threaded."""
    global pbar
    pbar = None
for i in raw_clips:
    os.makedirs(osp.join("/mnt/data/DataSet/K-RadarOOC/train/",str(i),"os2-64"),exist_ok = True)
    lidarfiles = glob.glob("/mnt/Kradar/K-Radar/{}/os2-64/*.pcd".format(i))
    with Pool(initializer=initializer, processes=8) as pool:
        # Use a chunksize of 1 to update the progress bar immediately for each task completed
        for _ in tqdm(pool.imap_unordered(from_pcd_to_bin, lidarfiles, chunksize=1), total=len(lidarfiles)):
            pass

100%|██████████| 1198/1198 [02:52<00:00,  6.93it/s]


In [27]:
import numpy as np
def from_npy_to_bin(file):
    array = np.load(file)
    # assert array.shape[0] > 0
    array=array.astype(np.float32)
    array.tofile(file+'.bin')
    # np.save(file,array)

In [28]:
import glob
from multiprocess import Pool, cpu_count
from tqdm import tqdm
import os
raw_clips = sorted(os.listdir('/mnt/Kradar/K-Radar'))
# Seq = [11,2,3,4,6,8,9,10,12,13,14,19,1,15]
def initializer():
    """Prevent tqdm from being multi-threaded."""
    global pbar
    pbar = None
for i in raw_clips:
    lidarfiles = glob.glob("/mnt/data/DataSet/K-RadarOOC/train/{}/radar_pc/*.npy".format(i))
    with Pool(initializer=initializer, processes=8) as pool:
        # Use a chunksize of 1 to update the progress bar immediately for each task completed
        for _ in tqdm(pool.imap_unordered(from_npy_to_bin, lidarfiles, chunksize=1), total=len(lidarfiles)):
            pass

100%|██████████| 1229/1229 [00:02<00:00, 541.40it/s]


In [7]:
data = np.load('/mnt/data/DataSet/K-RadarOOC/train/1/radar_pc/rpc_00001.npy')
data.dtype

dtype('float64')

In [ ]:
import scipy.io
mat_data = scipy.io.loadmat('/mnt/data/DataSet/K-Radar/1/radar_zyx_cube/cube_00001.mat')
variable_names = list(mat_data.keys())

np.shape(mat_data['arr_zyx'])
np.unique(mat_data['arr_zyx'])

In [ ]:
import glob
from multiprocess import Pool, cpu_count
from tqdm import tqdm
lidarfiles = glob.glob("/mnt/data/DataSet/K-Radar/1/os2-64/*.pcd")

def initializer():
    """Prevent tqdm from being multi-threaded."""
    global pbar
    pbar = None
with Pool(initializer=initializer, processes=8) as pool:
        # Use a chunksize of 1 to update the progress bar immediately for each task completed
        for _ in tqdm(pool.imap_unordered(from_pcd_to_bin, lidarfiles, chunksize=1), total=len(lidarfiles)):
            pass

In [35]:
len_radar = len(glob.glob("/mnt/data/DataSet/K-Radar/1/radar_zyx_cube/cube_*.mat"))
len_radar

629

In [2]:
import glob
import numpy as np
kradar_dict = {}
kradar_dict['metadata'] = {'version': 'kradar'}
kradar_dict['infos'] = []
training_len = 0
for seq in [4,6,7,8,9,10,11,12,13,14,16,17,18,19,20,27,56]:
# for seq in [1,15]:

    with open('/mnt/Kradar/K-Radar/{}/info_calib/calib_radar_lidar.txt'.format(seq)) as f:
                lines = f.readlines()
                f.close()
    list_calib = list(map(lambda x: float(x), lines[1].split(',')))
    frame_difference = int(list_calib[0])
    len_occgt = len(glob.glob("/mnt/data/DataSet/K-RadarOOC/train/{}/semantic_occupancy_gt_fov/occupancy_gt_with_semantic_fov*.npy".format(seq)))
    len_radar = len(glob.glob("/mnt/Kradar/K-Radar/{}/radar_zyx_cube/cube_*.mat".format(seq)))-frame_difference
    seq_size = min(len_occgt,len_radar)

    image_files = sorted(glob.glob('/mnt/Kradar/K-Radar/'+str(seq)+'/cam-front/'+ '*.png'))[::3]
    seq_size = min(seq_size,len(image_files))
    print(seq_size)
    training_len += seq_size
    for i in range(0,seq_size):
        seq_dict = {}
        seq_dict['occ_path'] = "/mnt/data/DataSet/K-RadarOOC/train/{}/semantic_occupancy_gt_fov/occupancy_gt_with_semantic_fov{}.npy".format(seq,i)
        seq_dict['lidar_path'] ="/mnt/data/DataSet/K-RadarOOC/train/"+str(seq)+"/radar_pc/rpc_{0:05d}.npy.bin".format(i+1+frame_difference)
        seq_dict['radar_path'] = "/mnt/Kradar/K-Radar/"+str(seq)+"/radar_zyx_cube/cube_{0:05d}.mat".format(i+1+frame_difference)
        seq_dict['radar_tensor_path'] = "/mnt/Kradar/K-Radar/"+str(seq)+"/radar_tesseract/tesseract_{0:05d}.mat".format(i+1+frame_difference)
        seq_dict['radar_polar_path'] = "/mnt/18T-Data/kradar/"+str(seq)+"/radar_polar_cube/DRAE_{0:05d}.npy".format(i+1+frame_difference)
        seq_dict['sparse_radar_path'] = "/mnt/18T-Data/kradar/"+str(seq)+"/radar_tensor_percentile/EAsparse_{0:05d}.npz".format(i+1+frame_difference)
        seq_dict['scene_token'] = seq
        seq_dict['lidar_token'] = i
        seq_dict['cams'] = {'CAM_FRONT': {'data_path': image_files[i],
'cam_intrinsic': np.array([[567.72077648,   0.        , 628.72078   ],
       [  0.        , 577.21369171, 369.30687   ],
       [  0.        ,   0.        ,   1.        ]])
,

                                'lidar2cam': np.array([[ 1.64325268e-04, -9.99390895e-01,  3.48971584e-02, 1.80000000e-01],
       [ 6.72339597e-03,  3.48974743e-02,  9.99368282e-01, -1.30000000e-01],
       [-9.99977384e-01,  7.04059534e-05,  6.72503525e-03, -3.12250226e-17],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]),
                                'sensor2lidar_translation': np.array([-1.80000000e-01,  1.30000000e-01,  3.12250226e-17]),
                                'sensor2lidar_rotation' : np.array([[ 1.64325268e-04,  6.72339597e-03, -9.99977384e-01],
       [-9.99390895e-01,  3.48974743e-02,  7.04059534e-05],
       [ 3.48971584e-02,  9.99368282e-01,  6.72503525e-03]])
                                        }
                            }
        seq_dict['lidar2ego_translation'] = [0,0,0]
        seq_dict['lidar2ego_rotation'] = [1,0,0,0]
        seq_dict['ego2global_translation'] = [0,0,0]
        seq_dict['ego2global_rotation'] = [1,0,0,0]
        seq_dict['timestamp'] = i * 1e16
        seq_dict['prev'] = i - 1
        seq_dict['next'] = i + 1
        seq_dict['lidarseg'] = None
        seq_dict['token'] = seq_dict['lidar_path']
        kradar_dict['infos'].append(seq_dict)
print("total training len {}".format(training_len))

588
594
595
567
833
1130
1195
888
227
595
578
593
594
592
595
597
572
total training len 11333


In [3]:
import pickle
with open('kradar_rpc_dict_train.pkl', 'wb') as f:
    pickle.dump(kradar_dict, f)

In [4]:
import glob
import numpy as np
kradar_dict = {}
kradar_dict['metadata'] = {'version': 'kradar'}
kradar_dict['infos'] = []
training_len = 0
for seq in [1,2]:
# for seq in [1,15]:

    with open('/mnt/Kradar/K-Radar/{}/info_calib/calib_radar_lidar.txt'.format(seq)) as f:
                lines = f.readlines()
                f.close()
    list_calib = list(map(lambda x: float(x), lines[1].split(',')))
    frame_difference = int(list_calib[0])
    len_occgt = len(glob.glob("/mnt/data/DataSet/K-RadarOOC/train/{}/semantic_occupancy_gt_fov/occupancy_gt_with_semantic_fov*.npy".format(seq)))
    len_radar = len(glob.glob("/mnt/Kradar/K-Radar/{}/radar_zyx_cube/cube_*.mat".format(seq)))-frame_difference
    seq_size = min(len_occgt,len_radar)

    image_files = sorted(glob.glob('/mnt/Kradar/K-Radar/'+str(seq)+'/cam-front/'+ '*.png'))[::3]
    seq_size = min(seq_size,len(image_files))
    print(seq_size)
    training_len += seq_size
    for i in range(0,seq_size):
        seq_dict = {}
        seq_dict['occ_path'] = "/mnt/data/DataSet/K-RadarOOC/train/{}/semantic_occupancy_gt_fov/occupancy_gt_with_semantic_fov{}.npy".format(seq,i)
        seq_dict['lidar_path'] ="/mnt/data/DataSet/K-RadarOOC/train/"+str(seq)+"/radar_pc/rpc_{0:05d}.npy.bin".format(i+1)
        seq_dict['radar_path'] = "/mnt/Kradar/K-Radar/"+str(seq)+"/radar_zyx_cube/cube_{0:05d}.mat".format(i+1+frame_difference)
        seq_dict['radar_tensor_path'] = "/mnt/Kradar/K-Radar/"+str(seq)+"/radar_tesseract/tesseract_{0:05d}.mat".format(i+1+frame_difference)
        seq_dict['radar_polar_path'] = "/mnt/18T-Data/kradar/"+str(seq)+"/radar_polar_cube/DRAE_{0:05d}.npy".format(i+1+frame_difference)
        seq_dict['sparse_radar_path'] = "/mnt/18T-Data/kradar/"+str(seq)+"/radar_tensor_percentile/EAsparse_{0:05d}.npz".format(i+1+frame_difference)
        seq_dict['scene_token'] = seq
        seq_dict['lidar_token'] = i
        seq_dict['cams'] = {'CAM_FRONT': {'data_path': image_files[i],
'cam_intrinsic': np.array([[567.72077648,   0.        , 628.72078   ],
       [  0.        , 577.21369171, 369.30687   ],
       [  0.        ,   0.        ,   1.        ]])
,

                                'lidar2cam': np.array([[ 1.64325268e-04, -9.99390895e-01,  3.48971584e-02, 1.80000000e-01],
       [ 6.72339597e-03,  3.48974743e-02,  9.99368282e-01, -1.30000000e-01],
       [-9.99977384e-01,  7.04059534e-05,  6.72503525e-03, -3.12250226e-17],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]),
                                'sensor2lidar_translation': np.array([-1.80000000e-01,  1.30000000e-01,  3.12250226e-17]),
                                'sensor2lidar_rotation' : np.array([[ 1.64325268e-04,  6.72339597e-03, -9.99977384e-01],
       [-9.99390895e-01,  3.48974743e-02,  7.04059534e-05],
       [ 3.48971584e-02,  9.99368282e-01,  6.72503525e-03]])
                                        }
                            }
        seq_dict['lidar2ego_translation'] = [0,0,0]
        seq_dict['lidar2ego_rotation'] = [1,0,0,0]
        seq_dict['ego2global_translation'] = [0,0,0]
        seq_dict['ego2global_rotation'] = [1,0,0,0]
        seq_dict['timestamp'] = i * 1e16
        seq_dict['prev'] = i - 1
        seq_dict['next'] = i + 1
        seq_dict['lidarseg'] = None
        seq_dict['token'] = seq_dict['lidar_path']
        kradar_dict['infos'].append(seq_dict)
print("total val len {}".format(training_len))

597
462
total val len 1059


In [34]:
11333 + 1059

12392

In [5]:
import pickle
with open('kradar_rpc_dict_val.pkl', 'wb') as f:
    pickle.dump(kradar_dict, f)

In [7]:
import glob
import numpy as np
kradar_dict = {}
kradar_dict['metadata'] = {'version': 'kradar'}
kradar_dict['infos'] = []
training_len = 0
# for seq in [1,2,4,6,7,8,9,10,11,12,13,14,16,17,18,19,20,27,56]:
for seq in [3,15,23,55,22]:

    with open('/mnt/Kradar/K-Radar/{}/info_calib/calib_radar_lidar.txt'.format(seq)) as f:
                lines = f.readlines()
                f.close()
    list_calib = list(map(lambda x: float(x), lines[1].split(',')))
    frame_difference = int(list_calib[0])
    len_occgt = len(glob.glob("/mnt/data/DataSet/K-RadarOOC/train/{}/semantic_occupancy_gt_fov/occupancy_gt_with_semantic_fov*.npy".format(seq)))
    len_radar = len(glob.glob("/mnt/Kradar/K-Radar/{}/radar_zyx_cube/cube_*.mat".format(seq)))-frame_difference
    seq_size = min(len_occgt,len_radar)

    image_files = sorted(glob.glob('/mnt/Kradar/K-Radar/'+str(seq)+'/cam-front/'+ '*.png'))[::3]
    seq_size = min(seq_size,len(image_files))
    print(seq_size)
    training_len += seq_size
    for i in range(0,seq_size):
        seq_dict = {}
        seq_dict['occ_path'] = "/mnt/data/DataSet/K-RadarOOC/train/{}/semantic_occupancy_gt_fov/occupancy_gt_with_semantic_fov{}.npy".format(seq,i)
        seq_dict['lidar_path'] ="/mnt/data/DataSet/K-RadarOOC/train/"+str(seq)+"/radar_pc/rpc_{0:05d}.npy.bin".format(i+1)
        seq_dict['radar_path'] = "/mnt/Kradar/K-Radar/"+str(seq)+"/radar_zyx_cube/cube_{0:05d}.mat".format(i+1+frame_difference)
        seq_dict['radar_tensor_path'] = "/mnt/Kradar/K-Radar/"+str(seq)+"/radar_tesseract/tesseract_{0:05d}.mat".format(i+1+frame_difference)
        seq_dict['radar_polar_path'] = "/mnt/18T-Data/kradar/"+str(seq)+"/radar_polar_cube/DRAE_{0:05d}.npy".format(i+1+frame_difference)
        seq_dict['sparse_radar_path'] = "/mnt/18T-Data/kradar/"+str(seq)+"/radar_tensor_percentile/EAsparse_{0:05d}.npz".format(i+1+frame_difference)
        seq_dict['scene_token'] = seq
        seq_dict['lidar_token'] = i
        seq_dict['cams'] = {'CAM_FRONT': {'data_path': image_files[i],
                                          
'cam_intrinsic': np.array([[567.72077648,   0.        , 628.72078   ],
       [  0.        , 577.21369171, 369.30687   ],
       [  0.        ,   0.        ,   1.        ]]),
                                'lidar2cam': np.array([[ 1.64325268e-04, -9.99390895e-01,  3.48971584e-02, 1.80000000e-01],
       [ 6.72339597e-03,  3.48974743e-02,  9.99368282e-01, -1.30000000e-01],
       [-9.99977384e-01,  7.04059534e-05,  6.72503525e-03, -3.12250226e-17],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]),
                                'sensor2lidar_translation': np.array([-1.80000000e-01,  1.30000000e-01,  3.12250226e-17]),
                                'sensor2lidar_rotation' : np.array([[ 1.64325268e-04,  6.72339597e-03, -9.99977384e-01],
       [-9.99390895e-01,  3.48974743e-02,  7.04059534e-05],
       [ 3.48971584e-02,  9.99368282e-01,  6.72503525e-03]]),
                                       'ego2global_translation' : [0,0,0],
                                       'ego2global_rotation' : [1,0,0,0]
                                       
                         
                                
                              
                                        }
                            }
        seq_dict['lidar2ego_translation'] = [0,0,0]
        seq_dict['lidar2ego_rotation'] = [1,0,0,0]
        seq_dict['ego2global_translation'] = [0,0,0]
        seq_dict['ego2global_rotation'] = [1,0,0,0]
        seq_dict['timestamp'] = i * 1e16
        seq_dict['prev'] = i - 1
        seq_dict['next'] = i + 1
        seq_dict['lidarseg'] = None
        seq_dict['token'] = seq_dict['lidar_path']
        kradar_dict['infos'].append(seq_dict)
print("total val len {}".format(training_len))

597
591
598
494
598
total val len 2878


In [8]:
import pickle
with open('kradar_rpc_dict_test.pkl', 'wb') as f:
    pickle.dump(kradar_dict, f)

In [7]:
len(kradar_dict['infos'])

7669

In [23]:
import glob
kradar_dict = {}
kradar_dict['metadata'] = {'version': 'kradar'}
kradar_dict['infos'] = []
# for seq in [11,2,3,4,6,8,9,10,12,13,14,19]:
for seq in [1,15]:

    with open('/mnt/data/DataSet/K-Radar/{}/info_calib/calib_radar_lidar.txt'.format(seq)) as f:
                lines = f.readlines()
                f.close()
    list_calib = list(map(lambda x: float(x), lines[1].split(',')))
    frame_difference = int(list_calib[0])
    len_occgt = len(glob.glob("/mnt/data/DataSet/K-RadarOOC/train/{}/semantic_occupancy_gt_fov/occupancy_gt_with_semantic_fov*.npy".format(seq)))
    len_radar = len(glob.glob("/mnt/data/DataSet/K-Radar/{}/radar_zyx_cube/cube_*.mat".format(seq)))-frame_difference
    seq_size = min(len_occgt,len_radar)
    print(seq_size)
    for i in range(0,seq_size):
        seq_dict = {}
        seq_dict['occ_path'] = "/mnt/data/DataSet/K-RadarOOC/train/{}/semantic_occupancy_gt_fov/occupancy_gt_with_semantic_fov{}.npy".format(seq,i)
        seq_dict['lidar_path'] ="/mnt/data/DataSet/K-Radar/"+str(seq)+"/radar_pc/rpc_{0:05d}.npy".format(i+1+frame_difference)
        seq_dict['radar_path'] = "/mnt/data/DataSet/K-Radar/"+str(seq)+"/radar_zyx_cube/cube_{0:05d}.mat".format(i+1+frame_difference)
        seq_dict['scene_token'] = seq
        seq_dict['lidar_token'] = i
        seq_dict['lidar2ego_translation'] = [0,0,0]
        seq_dict['lidar2ego_rotation'] = [1,0,0,0]
        seq_dict['ego2global_translation'] = [0,0,0]
        seq_dict['ego2global_rotation'] = [1,0,0,0]
        seq_dict['timestamp'] = i * 1e16
        seq_dict['prev'] = i - 1
        seq_dict['next'] = i + 1
        seq_dict['lidarseg'] = None
        seq_dict['token'] = seq_dict['lidar_path']

        kradar_dict['infos'].append(seq_dict)


597
591


In [24]:
import pickle
with open('kradar_radarpc_val.pkl', 'wb') as f:
    pickle.dump(kradar_dict, f)

In [13]:
array = np.load('/mnt/data/DataSet/K-Radar/1/radar_pc/rpc_00001.npy')

In [3]:
import glob
import numpy as np
kradar_dict = {}
kradar_dict['metadata'] = {'version': 'kradar'}
kradar_dict['infos'] = []
training_len = 0
# for seq in [1,2,4,6,7,8,9,10,11,12,13,14,16,17,18,19,20,27,56]:
for seq in [13]:

    with open('/mnt/Kradar/K-Radar/{}/info_calib/calib_radar_lidar.txt'.format(seq)) as f:
                lines = f.readlines()
                f.close()
    list_calib = list(map(lambda x: float(x), lines[1].split(',')))
    frame_difference = int(list_calib[0])
    len_occgt = len(glob.glob("/mnt/data/DataSet/K-RadarOOC/train/{}/semantic_occupancy_gt_fov/occupancy_gt_with_semantic_fov*.npy".format(seq)))
    len_radar = len(glob.glob("/mnt/Kradar/K-Radar/{}/radar_zyx_cube/cube_*.mat".format(seq)))-frame_difference
    seq_size = min(len_occgt,len_radar)

    image_files = sorted(glob.glob('/mnt/Kradar/K-Radar/'+str(seq)+'/cam-front/'+ '*.png'))[::3]
    seq_size = min(seq_size,len(image_files))
    print(len_occgt)
    training_len += seq_size
    for i in range(0,seq_size):
        seq_dict = {}
        seq_dict['occ_path'] = "/mnt/data/DataSet/K-RadarOOC/train/{}/semantic_occupancy_gt_fov/occupancy_gt_with_semantic_fov{}.npy".format(seq,i)
        seq_dict['lidar_path'] ="/mnt/data/DataSet/K-RadarOOC/train/"+str(seq)+"/radar_pc/rpc_{0:05d}.npy.bin".format(i+1)
        seq_dict['radar_path'] = "/mnt/Kradar/K-Radar/"+str(seq)+"/radar_zyx_cube/cube_{0:05d}.mat".format(i+1+frame_difference)
        seq_dict['radar_tensor_path'] = "/mnt/Kradar/K-Radar/"+str(seq)+"/radar_tesseract/tesseract_{0:05d}.mat".format(i+1+frame_difference)
        seq_dict['radar_polar_path'] = "/mnt/18T-Data/kradar/"+str(seq)+"/radar_polar_cube/DRAE_{0:05d}.npy".format(i+1+frame_difference)
        seq_dict['sparse_radar_path'] = "/mnt/18T-Data/kradar/"+str(seq)+"/radar_tensor_8doppler/EAsparse_{0:05d}.npz".format(i+1+frame_difference)
        seq_dict['scene_token'] = seq
        seq_dict['lidar_token'] = i
        seq_dict['cams'] = {'CAM_FRONT': {'data_path': image_files[i],
                                          
'cam_intrinsic': np.array([[567.72077648,   0.        , 628.72078   ],
       [  0.        , 577.21369171, 369.30687   ],
       [  0.        ,   0.        ,   1.        ]]),
                                'lidar2cam': np.array([[ 1.64325268e-04, -9.99390895e-01,  3.48971584e-02, 1.80000000e-01],
       [ 6.72339597e-03,  3.48974743e-02,  9.99368282e-01, -1.30000000e-01],
       [-9.99977384e-01,  7.04059534e-05,  6.72503525e-03, -3.12250226e-17],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]),
                                'sensor2lidar_translation': np.array([-1.80000000e-01,  1.30000000e-01,  3.12250226e-17]),
                                'sensor2lidar_rotation' : np.array([[ 1.64325268e-04,  6.72339597e-03, -9.99977384e-01],
       [-9.99390895e-01,  3.48974743e-02,  7.04059534e-05],
       [ 3.48971584e-02,  9.99368282e-01,  6.72503525e-03]]),
                                       'ego2global_translation' : [0,0,0],
                                       'ego2global_rotation' : [1,0,0,0]
                                       
                         
                                
                              
                                        }
                            }
        seq_dict['lidar2ego_translation'] = [0,0,0]
        seq_dict['lidar2ego_rotation'] = [1,0,0,0]
        seq_dict['ego2global_translation'] = [0,0,0]
        seq_dict['ego2global_rotation'] = [1,0,0,0]
        seq_dict['timestamp'] = i * 1e16
        seq_dict['prev'] = i - 1
        seq_dict['next'] = i + 1
        seq_dict['lidarseg'] = None
        seq_dict['token'] = seq_dict['lidar_path']
        kradar_dict['infos'].append(seq_dict)
print("total val len {}".format(training_len))

227
total val len 227


In [5]:
import glob
import numpy as np
kradar_dict = {}
kradar_dict['metadata'] = {'version': 'kradar'}
kradar_dict['infos'] = []
training_len = 0
for seq in [21,46,54]:
# for seq in [1,15]:

    with open('/mnt/Kradar/K-Radar/{}/info_calib/calib_radar_lidar.txt'.format(seq)) as f:
                lines = f.readlines()
                f.close()
    list_calib = list(map(lambda x: float(x), lines[1].split(',')))
    frame_difference = int(list_calib[0])
    len_occgt = len(glob.glob("/mnt/data/DataSet/K-RadarOOC/train/{}/semantic_occupancy_gt_fov/occupancy_gt_with_semantic_fov*.npy".format(seq)))
    len_radar = len(glob.glob("/mnt/Kradar/K-Radar/{}/radar_zyx_cube/cube_*.mat".format(seq)))-frame_difference
    seq_size = min(len_occgt,len_radar)

    image_files = sorted(glob.glob('/mnt/Kradar/K-Radar/'+str(seq)+'/cam-front/'+ '*.png'))[::3]
    seq_size = min(seq_size,len(image_files))
    print(seq_size)
    training_len += seq_size
    for i in range(0,seq_size):
        seq_dict = {}
        seq_dict['occ_path'] = "/mnt/data/DataSet/K-RadarOOC/train/{}/semantic_occupancy_gt_fov/occupancy_gt_with_semantic_fov{}.npy".format(seq,i)
        seq_dict['lidar_path'] ="/mnt/data/DataSet/K-RadarOOC/train/"+str(seq)+"/os2-32/os2-32_{0:05d}.bin".format(i+1)
        seq_dict['radar_path'] = "/mnt/Kradar/K-Radar/"+str(seq)+"/radar_zyx_cube/cube_{0:05d}.mat".format(i+1+frame_difference)
        seq_dict['radar_tensor_path'] = "/mnt/Kradar/K-Radar/"+str(seq)+"/radar_tesseract/tesseract_{0:05d}.mat".format(i+1+frame_difference)
        seq_dict['radar_polar_path'] = "/mnt/18T-Data/kradar/"+str(seq)+"/radar_polar_cube/DRAE_{0:05d}.npy".format(i+1+frame_difference)
        seq_dict['sparse_radar_path'] = "/mnt/18T-Data/kradar/"+str(seq)+"/radar_tensor_8doppler/EAsparse_{0:05d}.npz".format(i+1+frame_difference)
        seq_dict['scene_token'] = seq
        seq_dict['lidar_token'] = i
        seq_dict['cams'] = {'CAM_FRONT': {'data_path': image_files[i],
'cam_intrinsic': np.array([[567.72077648,   0.        , 628.72078   ],
       [  0.        , 577.21369171, 369.30687   ],
       [  0.        ,   0.        ,   1.        ]])
,

                                'lidar2cam': np.array([[ 1.64325268e-04, -9.99390895e-01,  3.48971584e-02, 1.80000000e-01],
       [ 6.72339597e-03,  3.48974743e-02,  9.99368282e-01, -1.30000000e-01],
       [-9.99977384e-01,  7.04059534e-05,  6.72503525e-03, -3.12250226e-17],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]),
                                'sensor2lidar_translation': np.array([-1.80000000e-01,  1.30000000e-01,  3.12250226e-17]),
                                'sensor2lidar_rotation' : np.array([[ 1.64325268e-04,  6.72339597e-03, -9.99977384e-01],
       [-9.99390895e-01,  3.48974743e-02,  7.04059534e-05],
       [ 3.48971584e-02,  9.99368282e-01,  6.72503525e-03]])
                                        }
                            }
        seq_dict['lidar2ego_translation'] = [0,0,0]
        seq_dict['lidar2ego_rotation'] = [1,0,0,0]
        seq_dict['ego2global_translation'] = [0,0,0]
        seq_dict['ego2global_rotation'] = [1,0,0,0]
        seq_dict['timestamp'] = i * 1e16
        seq_dict['prev'] = i - 1
        seq_dict['next'] = i + 1
        seq_dict['lidarseg'] = None
        seq_dict['token'] = seq_dict['lidar_path']
        kradar_dict['infos'].append(seq_dict)
print("total training len {}".format(training_len))

597
579
573
total training len 1749


In [6]:
import pickle
with open('adv_weather.pkl', 'wb') as f:
    pickle.dump(kradar_dict, f)

In [1]:
import pandas as pd
pd.read_pickle('kradar_dict_test.pkl')

{'metadata': {'version': 'kradar'},
 'infos': [{'occ_path': '/mnt/data/DataSet/K-RadarOOC/train/3/semantic_occupancy_gt_fov/occupancy_gt_with_semantic_fov0.npy',
   'lidar_path': '/mnt/data/DataSet/K-RadarOOC/train/3/os2-64/os2-64_00001.pcd.bin',
   'radar_path': '/mnt/Kradar/K-Radar/3/radar_zyx_cube/cube_00031.mat',
   'radar_tensor_path': '/mnt/Kradar/K-Radar/3/radar_tesseract/tesseract_00031.mat',
   'radar_polar_path': '/mnt/12T-Data/Kradar/3/radar_polar_cube/cube_00031.npy',
   'scene_token': 3,
   'lidar_token': 0,
   'cams': {'CAM_FRONT': {'data_path': '/mnt/Kradar/K-Radar/3/cam-front/cam-front_00001.png',
     'cam_intrinsic': array([[567.72077648,   0.        , 628.72078   ],
            [  0.        , 577.21369171, 369.30687   ],
            [  0.        ,   0.        ,   1.        ]]),
     'lidar2cam': array([[ 1.64325268e-04, -9.99390895e-01,  3.48971584e-02,
              1.80000000e-01],
            [ 6.72339597e-03,  3.48974743e-02,  9.99368282e-01,
             -1.3000